In [23]:
import os

In [34]:
%pwd

'D:\\nlp\\text-summarizer'

In [33]:
os.chdir("D:\\nlp\\text-summarizer")

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir: Path

In [36]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [37]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [38]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [39]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-17 11:58:04,609: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-17 11:58:04,614: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-17 11:58:04,619: INFO: common: created directory at: artifacts]
[2025-04-17 11:58:04,623: INFO: common: created directory at: artifacts/data_ingestion]


[2025-04-17 11:59:29,491: INFO: 1434958058: artifacts/data_ingestion/samsumdata.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 68C0:219CD5:DCB8B:21A540:68009F78
Accept-Ranges: bytes
Date: Thu, 17 Apr 2025 06:28:10 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4734-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1744871289.629471,VS0,VE2056
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 73001aa9501a1d8db06ab2cb04fe0ca065be29e9
Expires: Thu, 17 Apr 2025 06:33:10 GMT
Source-Age: 2

]
